In [13]:
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical, np_utils
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D
from keras.models import Model, model_from_json
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
import os
import pandas as pd
import librosa
import glob 
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder


In [14]:
mylist= os.listdir(r'E:/SpeechEmotion/')
feeling_list=[]
for item in mylist:
    if item[3:-19]=='01':
        if item[6:-16]=='03' and int(item[18:-4])%2==0:
            feeling_list.append('female_happy')
        elif item[6:-16]=='03' and int(item[18:-4])%2==1:
            feeling_list.append('male_happy')
        elif item[6:-16]=='04' and int(item[18:-4])%2==0:
            feeling_list.append('female_sad')
        elif item[6:-16]=='04' and int(item[18:-4])%2==1:
            feeling_list.append('male_sad')
        elif item[6:-16]=='05' and int(item[18:-4])%2==0:
            feeling_list.append('female_angry')
        elif item[6:-16]=='05' and int(item[18:-4])%2==1:
            feeling_list.append('male_angry')

        
    

In [15]:
labels = pd.DataFrame(feeling_list)
df = pd.DataFrame(columns=['feature'])
bookmark=0
for index,y in enumerate(mylist):
    if mylist[index][3:-19]=='01' and (mylist[index][6:-16]=='03' or mylist[index][6:-16]=='04' or 
                                       mylist[index][6:-16]=='05'):
        X, sample_rate = librosa.load('E:/SpeechEmotion/'+y, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
        sample_rate = np.array(sample_rate)
        mfccs = np.mean(librosa.feature.mfcc(y=X, 
                                            sr=sample_rate, 
                                            n_mfcc=13),
                        axis=0)
        feature = mfccs
        df.loc[bookmark] = [feature]
        bookmark=bookmark+1
        

In [16]:
df3 = pd.DataFrame(df['feature'].values.tolist())
newdf = pd.concat([df3,labels], axis=1)
rnewdf = newdf.rename(index=str, columns={"0": "label"})
rnewdf = shuffle(newdf)
rnewdf=rnewdf.fillna(0)
newdf1 = np.random.rand(len(rnewdf)) < 0.8
train = rnewdf[newdf1]
test = rnewdf[~newdf1]
trainfeatures = train.iloc[:, :-1]
trainlabel = train.iloc[:, -1:]
testfeatures = test.iloc[:, :-1]
testlabel = test.iloc[:, -1:]

In [17]:
X_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
X_test = np.array(testfeatures)
y_test = np.array(testlabel)
lb = LabelEncoder()

y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

x_traincnn =np.expand_dims(X_train, axis=2)
x_testcnn= np.expand_dims(X_test, axis=2)


In [18]:
model = Sequential()

model.add(Conv1D(256, 5,padding='same',
                 input_shape=(216,1)))
model.add(Activation('relu'))
model.add(Conv1D(128, 5,padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(8)))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Conv1D(128, 5,padding='same',))
#model.add(Activation('relu'))
#model.add(Dropout(0.2))
model.add(Conv1D(128, 5,padding='same',))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(6))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, 216, 256)          1536      
_________________________________________________________________
activation_11 (Activation)   (None, 216, 256)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 216, 128)          163968    
_________________________________________________________________
activation_12 (Activation)   (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 27, 128)          

In [19]:
model.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
checkpoint = ModelCheckpoint(r"E:\demo1\speeash14.h5", monitor='val_accuracy',save_weights_only=True, mode='max', 
                             verbose=1,
                             save_best_only=True)
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=1000, validation_data=(x_testcnn, y_test),
                     callbacks=[checkpoint])

fer_json = model.to_json()
with open("E:\demo1\speeash14.json", "w") as json_file:
    json_file.write(fer_json)
  

Train on 468 samples, validate on 108 samples
Epoch 1/2100
468/468 [==============================] - ETA: 1:23 - loss: 2.0111 - accuracy: 0.25 - ETA: 41s - loss: 2.1947 - accuracy: 0.2188 - ETA: 28s - loss: 2.0644 - accuracy: 0.187 - ETA: 21s - loss: 2.0311 - accuracy: 0.171 - ETA: 16s - loss: 2.0173 - accuracy: 0.150 - ETA: 13s - loss: 2.0160 - accuracy: 0.145 - ETA: 11s - loss: 1.9836 - accuracy: 0.178 - ETA: 10s - loss: 1.9731 - accuracy: 0.171 - ETA: 8s - loss: 1.9613 - accuracy: 0.173 - ETA: 7s - loss: 1.9588 - accuracy: 0.16 - ETA: 6s - loss: 1.9564 - accuracy: 0.15 - ETA: 6s - loss: 1.9455 - accuracy: 0.15 - ETA: 5s - loss: 1.9412 - accuracy: 0.15 - ETA: 4s - loss: 1.9311 - accuracy: 0.15 - ETA: 4s - loss: 1.9249 - accuracy: 0.16 - ETA: 3s - loss: 1.9138 - accuracy: 0.16 - ETA: 3s - loss: 1.9066 - accuracy: 0.18 - ETA: 3s - loss: 1.9076 - accuracy: 0.17 - ETA: 2s - loss: 1.8985 - accuracy: 0.17 - ETA: 2s - loss: 1.8944 - accuracy: 0.17 - ETA: 2s - loss: 1.8932 - accuracy: 0.17 

468/468 [==============================] - ETA: 5s - loss: 1.7211 - accuracy: 0.18 - ETA: 5s - loss: 1.6449 - accuracy: 0.31 - ETA: 5s - loss: 1.6105 - accuracy: 0.35 - ETA: 5s - loss: 1.5972 - accuracy: 0.32 - ETA: 5s - loss: 1.6031 - accuracy: 0.36 - ETA: 4s - loss: 1.5924 - accuracy: 0.36 - ETA: 4s - loss: 1.5696 - accuracy: 0.38 - ETA: 3s - loss: 1.5664 - accuracy: 0.39 - ETA: 3s - loss: 1.5605 - accuracy: 0.40 - ETA: 3s - loss: 1.5635 - accuracy: 0.40 - ETA: 2s - loss: 1.5775 - accuracy: 0.38 - ETA: 2s - loss: 1.5762 - accuracy: 0.38 - ETA: 2s - loss: 1.5731 - accuracy: 0.39 - ETA: 2s - loss: 1.5781 - accuracy: 0.37 - ETA: 2s - loss: 1.5800 - accuracy: 0.36 - ETA: 1s - loss: 1.5807 - accuracy: 0.35 - ETA: 1s - loss: 1.5814 - accuracy: 0.34 - ETA: 1s - loss: 1.5729 - accuracy: 0.35 - ETA: 1s - loss: 1.5720 - accuracy: 0.36 - ETA: 1s - loss: 1.5816 - accuracy: 0.35 - ETA: 1s - loss: 1.5912 - accuracy: 0.33 - ETA: 0s - loss: 1.5879 - accuracy: 0.34 - ETA: 0s - loss: 1.5968 - accuracy

468/468 [==============================] - ETA: 6s - loss: 1.4115 - accuracy: 0.43 - ETA: 6s - loss: 1.4708 - accuracy: 0.37 - ETA: 6s - loss: 1.4669 - accuracy: 0.41 - ETA: 5s - loss: 1.4938 - accuracy: 0.35 - ETA: 5s - loss: 1.5171 - accuracy: 0.32 - ETA: 5s - loss: 1.4940 - accuracy: 0.33 - ETA: 5s - loss: 1.5013 - accuracy: 0.33 - ETA: 5s - loss: 1.5052 - accuracy: 0.33 - ETA: 4s - loss: 1.4993 - accuracy: 0.36 - ETA: 4s - loss: 1.4965 - accuracy: 0.36 - ETA: 4s - loss: 1.4924 - accuracy: 0.38 - ETA: 3s - loss: 1.4915 - accuracy: 0.37 - ETA: 3s - loss: 1.4822 - accuracy: 0.38 - ETA: 3s - loss: 1.4752 - accuracy: 0.37 - ETA: 3s - loss: 1.4782 - accuracy: 0.37 - ETA: 3s - loss: 1.4781 - accuracy: 0.36 - ETA: 2s - loss: 1.4785 - accuracy: 0.36 - ETA: 2s - loss: 1.4806 - accuracy: 0.36 - ETA: 2s - loss: 1.4784 - accuracy: 0.36 - ETA: 2s - loss: 1.4752 - accuracy: 0.36 - ETA: 1s - loss: 1.4743 - accuracy: 0.37 - ETA: 1s - loss: 1.4761 - accuracy: 0.37 - ETA: 1s - loss: 1.4785 - accuracy

468/468 [==============================] - ETA: 5s - loss: 1.4527 - accuracy: 0.43 - ETA: 5s - loss: 1.3974 - accuracy: 0.43 - ETA: 4s - loss: 1.3634 - accuracy: 0.45 - ETA: 4s - loss: 1.4627 - accuracy: 0.40 - ETA: 4s - loss: 1.4421 - accuracy: 0.42 - ETA: 4s - loss: 1.4620 - accuracy: 0.41 - ETA: 4s - loss: 1.4326 - accuracy: 0.44 - ETA: 4s - loss: 1.4357 - accuracy: 0.43 - ETA: 3s - loss: 1.4163 - accuracy: 0.44 - ETA: 3s - loss: 1.4083 - accuracy: 0.46 - ETA: 3s - loss: 1.3920 - accuracy: 0.48 - ETA: 3s - loss: 1.3927 - accuracy: 0.47 - ETA: 2s - loss: 1.3929 - accuracy: 0.46 - ETA: 2s - loss: 1.3960 - accuracy: 0.46 - ETA: 2s - loss: 1.4072 - accuracy: 0.46 - ETA: 2s - loss: 1.4008 - accuracy: 0.46 - ETA: 2s - loss: 1.3960 - accuracy: 0.45 - ETA: 1s - loss: 1.3861 - accuracy: 0.47 - ETA: 1s - loss: 1.3882 - accuracy: 0.47 - ETA: 1s - loss: 1.3933 - accuracy: 0.47 - ETA: 1s - loss: 1.3959 - accuracy: 0.46 - ETA: 1s - loss: 1.3965 - accuracy: 0.45 - ETA: 0s - loss: 1.3915 - accuracy

468/468 [==============================] - ETA: 7s - loss: 1.2058 - accuracy: 0.56 - ETA: 6s - loss: 1.3488 - accuracy: 0.46 - ETA: 6s - loss: 1.3377 - accuracy: 0.43 - ETA: 5s - loss: 1.3187 - accuracy: 0.43 - ETA: 5s - loss: 1.3154 - accuracy: 0.45 - ETA: 4s - loss: 1.3018 - accuracy: 0.44 - ETA: 4s - loss: 1.2852 - accuracy: 0.43 - ETA: 4s - loss: 1.2774 - accuracy: 0.44 - ETA: 4s - loss: 1.2916 - accuracy: 0.45 - ETA: 4s - loss: 1.2854 - accuracy: 0.45 - ETA: 3s - loss: 1.2778 - accuracy: 0.44 - ETA: 3s - loss: 1.2779 - accuracy: 0.44 - ETA: 3s - loss: 1.2668 - accuracy: 0.45 - ETA: 3s - loss: 1.2676 - accuracy: 0.46 - ETA: 3s - loss: 1.2623 - accuracy: 0.47 - ETA: 2s - loss: 1.2651 - accuracy: 0.47 - ETA: 2s - loss: 1.2679 - accuracy: 0.47 - ETA: 2s - loss: 1.2769 - accuracy: 0.47 - ETA: 2s - loss: 1.2770 - accuracy: 0.48 - ETA: 1s - loss: 1.2770 - accuracy: 0.47 - ETA: 1s - loss: 1.2806 - accuracy: 0.46 - ETA: 1s - loss: 1.2883 - accuracy: 0.46 - ETA: 1s - loss: 1.2876 - accuracy

...

468/468 [==============================] - ETA: 5s - loss: 0.0262 - accuracy: 1.00 - ETA: 5s - loss: 0.0226 - accuracy: 1.00 - ETA: 5s - loss: 0.0192 - accuracy: 1.00 - ETA: 5s - loss: 0.0165 - accuracy: 1.00 - ETA: 5s - loss: 0.0177 - accuracy: 1.00 - ETA: 4s - loss: 0.0158 - accuracy: 1.00 - ETA: 4s - loss: 0.0154 - accuracy: 1.00 - ETA: 4s - loss: 0.0150 - accuracy: 1.00 - ETA: 4s - loss: 0.0146 - accuracy: 1.00 - ETA: 4s - loss: 0.0143 - accuracy: 1.00 - ETA: 3s - loss: 0.0146 - accuracy: 1.00 - ETA: 3s - loss: 0.0137 - accuracy: 1.00 - ETA: 3s - loss: 0.0137 - accuracy: 1.00 - ETA: 3s - loss: 0.0140 - accuracy: 1.00 - ETA: 3s - loss: 0.0153 - accuracy: 1.00 - ETA: 2s - loss: 0.0156 - accuracy: 1.00 - ETA: 2s - loss: 0.0153 - accuracy: 1.00 - ETA: 2s - loss: 0.0151 - accuracy: 1.00 - ETA: 2s - loss: 0.0151 - accuracy: 1.00 - ETA: 1s - loss: 0.0147 - accuracy: 1.00 - ETA: 1s - loss: 0.0151 - accuracy: 1.00 - ETA: 1s - loss: 0.0147 - accuracy: 1.00 - ETA: 1s - loss: 0.0148 - accuracy

468/468 [==============================] - ETA: 5s - loss: 0.0078 - accuracy: 1.00 - ETA: 5s - loss: 0.0109 - accuracy: 1.00 - ETA: 5s - loss: 0.0092 - accuracy: 1.00 - ETA: 4s - loss: 0.0107 - accuracy: 1.00 - ETA: 4s - loss: 0.0095 - accuracy: 1.00 - ETA: 4s - loss: 0.0095 - accuracy: 1.00 - ETA: 4s - loss: 0.0117 - accuracy: 1.00 - ETA: 4s - loss: 0.0118 - accuracy: 1.00 - ETA: 3s - loss: 0.0117 - accuracy: 1.00 - ETA: 3s - loss: 0.0152 - accuracy: 1.00 - ETA: 3s - loss: 0.0140 - accuracy: 1.00 - ETA: 3s - loss: 0.0136 - accuracy: 1.00 - ETA: 3s - loss: 0.0141 - accuracy: 1.00 - ETA: 3s - loss: 0.0141 - accuracy: 1.00 - ETA: 2s - loss: 0.0139 - accuracy: 1.00 - ETA: 2s - loss: 0.0134 - accuracy: 1.00 - ETA: 2s - loss: 0.0131 - accuracy: 1.00 - ETA: 2s - loss: 0.0131 - accuracy: 1.00 - ETA: 2s - loss: 0.0129 - accuracy: 1.00 - ETA: 1s - loss: 0.0125 - accuracy: 1.00 - ETA: 1s - loss: 0.0129 - accuracy: 1.00 - ETA: 1s - loss: 0.0126 - accuracy: 1.00 - ETA: 1s - loss: 0.0125 - accuracy

468/468 [==============================] - ETA: 5s - loss: 0.0103 - accuracy: 1.00 - ETA: 5s - loss: 0.0113 - accuracy: 1.00 - ETA: 5s - loss: 0.0088 - accuracy: 1.00 - ETA: 4s - loss: 0.0085 - accuracy: 1.00 - ETA: 4s - loss: 0.0086 - accuracy: 1.00 - ETA: 4s - loss: 0.0086 - accuracy: 1.00 - ETA: 4s - loss: 0.0081 - accuracy: 1.00 - ETA: 4s - loss: 0.0079 - accuracy: 1.00 - ETA: 3s - loss: 0.0084 - accuracy: 1.00 - ETA: 3s - loss: 0.0082 - accuracy: 1.00 - ETA: 3s - loss: 0.0083 - accuracy: 1.00 - ETA: 3s - loss: 0.0079 - accuracy: 1.00 - ETA: 3s - loss: 0.0096 - accuracy: 1.00 - ETA: 2s - loss: 0.0102 - accuracy: 1.00 - ETA: 2s - loss: 0.0107 - accuracy: 1.00 - ETA: 2s - loss: 0.0104 - accuracy: 1.00 - ETA: 2s - loss: 0.0103 - accuracy: 1.00 - ETA: 2s - loss: 0.0108 - accuracy: 1.00 - ETA: 1s - loss: 0.0115 - accuracy: 1.00 - ETA: 1s - loss: 0.0122 - accuracy: 1.00 - ETA: 1s - loss: 0.0123 - accuracy: 1.00 - ETA: 1s - loss: 0.0121 - accuracy: 1.00 - ETA: 1s - loss: 0.0119 - accuracy

468/468 [==============================] - ETA: 4s - loss: 0.0080 - accuracy: 1.00 - ETA: 3s - loss: 0.0078 - accuracy: 1.00 - ETA: 3s - loss: 0.0074 - accuracy: 1.00 - ETA: 3s - loss: 0.0094 - accuracy: 1.00 - ETA: 2s - loss: 0.0095 - accuracy: 1.00 - ETA: 2s - loss: 0.0095 - accuracy: 1.00 - ETA: 2s - loss: 0.0090 - accuracy: 1.00 - ETA: 2s - loss: 0.0094 - accuracy: 1.00 - ETA: 2s - loss: 0.0087 - accuracy: 1.00 - ETA: 2s - loss: 0.0084 - accuracy: 1.00 - ETA: 2s - loss: 0.0084 - accuracy: 1.00 - ETA: 1s - loss: 0.0082 - accuracy: 1.00 - ETA: 1s - loss: 0.0087 - accuracy: 1.00 - ETA: 1s - loss: 0.0092 - accuracy: 1.00 - ETA: 1s - loss: 0.0090 - accuracy: 1.00 - ETA: 1s - loss: 0.0086 - accuracy: 1.00 - ETA: 1s - loss: 0.0085 - accuracy: 1.00 - ETA: 1s - loss: 0.0085 - accuracy: 1.00 - ETA: 1s - loss: 0.0085 - accuracy: 1.00 - ETA: 1s - loss: 0.0086 - accuracy: 1.00 - ETA: 0s - loss: 0.0084 - accuracy: 1.00 - ETA: 0s - loss: 0.0087 - accuracy: 1.00 - ETA: 0s - loss: 0.0087 - accuracy

KeyboardInterrupt: 

In [20]:
fer_json = model.to_json()
with open("E:\demo1\speeash15.json", "w") as json_file:#0-62% 12->60.396
    json_file.write(fer_json)